In [1]:
%load_ext sql
%sql duckdb:///../../database/data.duckdb

The 'toml' package isn't installed. To load settings from pyproject.toml or ~/.jupysql/config, install with: pip install toml

Connecting to 'duckdb:///../../database/data.duckdb'

In [2]:
%%sql
SELECT * FROM int__valeurs_de_reference WHERE categorie_1='nitrate'

Running query in 'duckdb:///../../database/data.duckdb'

cdparametresiseeaux,categorie_1,categorie_2,categorie_3,limite_qualite,limite_qualite_unite,valeur_sanitaire_1,valeur_sanitaire_1_unite,valeur_sanitaire_2,valeur_sanitaire_2_unite,nb_rows
NO2,nitrate,None,None,0.5,mg/L,None,None,None,None,1
NO3,nitrate,None,None,50.0,mg/L,None,None,None,None,1
NO3_NO2,nitrate,None,None,1.0,None,None,None,None,None,1


In [3]:
%%sql
SELECT
    MAX(
        CASE
            WHEN
                cdparametresiseeaux = 'NO3'
                THEN limite_qualite
        END
    ) AS limite_qualite_no3,
    MAX(
        CASE
            WHEN
                cdparametresiseeaux = 'NO3_NO2'
                THEN limite_qualite
        END
    ) AS limite_qualite_no3_no2,
    MAX(
        CASE
            WHEN
                cdparametresiseeaux = 'NO2'
                THEN limite_qualite
        END
    ) AS limite_qualite_no2
FROM
    int__valeurs_de_reference
WHERE
    categorie_1 = 'nitrate'

Running query in 'duckdb:///../../database/data.duckdb'

limite_qualite_no3,limite_qualite_no3_no2,limite_qualite_no2
50.0,1.0,0.5


**Les règles choisies**
    
_Si on a les 3 paramètres disponibles_
- Si Nitrates < 50 mg/L et nitrites < 0,5 mg/L et nitrate/50 + nitrites/3 < 1 mg/L  --> CONFORME
- Si Nitrates >= 50 mg/L et/ou nitrites >= 0,5 mg/L et/ou nitrate/50 + nitrites/3 >= 1 mg/L --> NON CONFORME


_Si on n'a pas  nitrate/50 + nitrites/3 mais qu'on a nitrate et  nitrites_<br>
==> on le calcule et on applique comme ci-dessus

- Si on n'a pas nitrate -->  donnee_manquante
- Si on n'a pas nitrites et que  nitrate >= 50 mg/L --> NON CONFORME
- Si on n'a pas nitrites et que  nitrate < 50 mg/L --> donnee_manquant 


In [4]:
%%sql --save int__resultats_nitrites_udi_dernier
WITH
last_pvl AS (
    SELECT
        cdreseau,
        categorie,
        cdparametresiseeaux,
        limite_qualite,
        datetimeprel,
        valtraduite,
        ROW_NUMBER()
            OVER (
                PARTITION BY cdreseau, cdparametresiseeaux
                ORDER BY datetimeprel DESC
            )
            AS row_number
    FROM
       int__resultats_udi_communes
    WHERE
        categorie = 'nitrate'
        AND
        -- On garde les prélèvements de moins d'un an
        CURRENT_DATE - datetimeprel < INTERVAL 1 YEAR
),

valeur_ref AS (
    SELECT
        MAX(
            CASE
                WHEN
                    cdparametresiseeaux = 'NO3'
                    THEN limite_qualite
            END
        ) AS limite_qualite_no3,
        MAX(
            CASE
                WHEN
                    cdparametresiseeaux = 'NO3_NO2'
                    THEN limite_qualite
            END
        ) AS limite_qualite_no3_no2,
        MAX(
            CASE
                WHEN
                    cdparametresiseeaux = 'NO2'
                    THEN limite_qualite
            END
        ) AS limite_qualite_no2
    FROM
        int__valeurs_de_reference
    WHERE
        categorie_1 = 'nitrate'
),

split_nitrites AS (
    SELECT
        last_pvl.cdreseau,
        last_pvl.categorie,
        COUNT(DISTINCT last_pvl.cdparametresiseeaux) AS nb_parametres,
        MAX(last_pvl.datetimeprel) AS dernier_prel_datetime,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO3'
                    THEN last_pvl.valtraduite
            END
        ) AS valtraduite_no3,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO2'
                    THEN last_pvl.valtraduite
            END
        ) AS valtraduite_no2,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO3_NO2'
                    THEN last_pvl.valtraduite
            END
        ) AS valtraduite_no3_no2
    FROM
        last_pvl
    WHERE
        last_pvl.row_number = 1
    GROUP BY
        last_pvl.cdreseau,
        last_pvl.categorie
),

split_nitrites_with_ref AS (
    SELECT
        split_nitrites.*,
        valeur_ref.limite_qualite_no3,
        valeur_ref.limite_qualite_no3_no2,
        valeur_ref.limite_qualite_no2,
        split_nitrites.valtraduite_no3 / 50
        + split_nitrites.valtraduite_no2 / 3 AS valtraduite_no3_no2_calc
    FROM split_nitrites
    CROSS JOIN valeur_ref

)

SELECT
    split_nitrites_with_ref.cdreseau,
    split_nitrites_with_ref.categorie,
    'dernier_prel' AS periode,
    split_nitrites_with_ref.dernier_prel_datetime,
    split_nitrites_with_ref.nb_parametres,
    CASE
        WHEN
            split_nitrites_with_ref.valtraduite_no3
            < split_nitrites_with_ref.limite_qualite_no3
            AND split_nitrites_with_ref.valtraduite_no2
            < split_nitrites_with_ref.limite_qualite_no2
            AND COALESCE(
                split_nitrites_with_ref.valtraduite_no3_no2,
                split_nitrites_with_ref.valtraduite_no3_no2_calc
            )
            < split_nitrites_with_ref.limite_qualite_no3_no2
            THEN 'conforme'
        WHEN
            split_nitrites_with_ref.valtraduite_no3
            >= split_nitrites_with_ref.limite_qualite_no3
            OR split_nitrites_with_ref.valtraduite_no2
            >= split_nitrites_with_ref.limite_qualite_no2
            OR COALESCE(
                split_nitrites_with_ref.valtraduite_no3_no2,
                split_nitrites_with_ref.valtraduite_no3_no2_calc
            )
            >= split_nitrites_with_ref.limite_qualite_no3_no2
            THEN 'non_conforme'
        WHEN
            split_nitrites_with_ref.valtraduite_no3 IS NULL
            OR (
                split_nitrites_with_ref.valtraduite_no2 IS NULL
                AND split_nitrites_with_ref.valtraduite_no3
                < split_nitrites_with_ref.limite_qualite_no3
            )
            THEN 'donnee_manquante'
        ELSE 'error'
    END AS resultat
FROM
    split_nitrites_with_ref


Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

cdreseau,categorie,periode,dernier_prel_datetime,nb_parametres,resultat
001000254,nitrate,dernier_prel,2024-12-26 10:15:00,3,conforme
001000285,nitrate,dernier_prel,2025-01-22 11:24:00,3,conforme
001000289,nitrate,dernier_prel,2025-01-21 11:14:00,3,conforme
001000359,nitrate,dernier_prel,2025-01-13 10:42:00,3,conforme
001000369,nitrate,dernier_prel,2025-02-19 09:12:00,3,conforme
001000408,nitrate,dernier_prel,2024-07-17 08:33:00,3,conforme
001000421,nitrate,dernier_prel,2024-08-26 09:44:00,3,conforme
001000493,nitrate,dernier_prel,2024-09-13 11:08:00,3,conforme
001000645,nitrate,dernier_prel,2024-12-03 09:52:00,3,conforme
001000715,nitrate,dernier_prel,2024-08-26 11:52:00,3,conforme


In [5]:
%%sql --with int__resultats_nitrites_udi_dernier
SELECT 
    resultat, COUNT(*)
FROM 
    int__resultats_nitrites_udi_dernier
GROUP BY 1

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

resultat,count_star()
donnee_manquante,1162
non_conforme,269
conforme,20704


In [6]:
%%sql --with int__resultats_nitrites_udi_dernier
SELECT 
    * 
FROM 
    int__resultats_nitrites_udi_dernier
WHERE 
    resultat='error'

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

cdreseau,categorie,periode,dernier_prel_datetime,nb_parametres,resultat


In [7]:
%%sql --with int__resultats_nitrites_udi_dernier
SELECT 
    resultat, MIN( cdreseau) min_cdreseau, MAX(cdreseau) AS max_cdreseau
FROM 
    int__resultats_nitrites_udi_dernier
GROUP BY 1

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

resultat,min_cdreseau,max_cdreseau
non_conforme,002000060,095000346
conforme,001000003,976003489
donnee_manquante,001000251,976003881


# Check 

In [8]:
%%sql
WITH
last_pvl AS (
    SELECT
        cdreseau,
        categorie,
        cdparametresiseeaux,
        limite_qualite,
        datetimeprel,
        valtraduite,
        ROW_NUMBER()
            OVER (
                PARTITION BY cdreseau, cdparametresiseeaux
                ORDER BY datetimeprel DESC
            )
            AS row_number
    FROM
       int__resultats_udi_communes
    WHERE
        categorie = 'nitrate'
        AND
        -- On garde les prélèvements de moins d'un an
        CURRENT_DATE - datetimeprel < INTERVAL 1 YEAR
),

valeur_ref AS (
SELECT 
    MAX(
            CASE
                WHEN
                    cdparametresiseeaux = 'NO3'
                    THEN limite_qualite
            END
        ) AS limite_qualite_no3,
        MAX(
            CASE
                WHEN
                    cdparametresiseeaux = 'NO3_NO2'
                    THEN limite_qualite
            END
        ) AS limite_qualite_no3_no2,
        MAX(
            CASE
                WHEN
                    cdparametresiseeaux = 'NO2'
                    THEN limite_qualite
            END
        ) AS limite_qualite_no2
FROM int__valeurs_de_reference
WHERE categorie_1='nitrate'    
),

split_nitrites AS (
    SELECT
        last_pvl.cdreseau,
        last_pvl.categorie,
        COUNT(DISTINCT last_pvl.cdparametresiseeaux) AS nb_parametres,
        MAX(last_pvl.datetimeprel) AS dernier_prel_datetime,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO3'
                    THEN last_pvl.datetimeprel
            END
        ) AS dernier_prel_datetime_n03,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO2'
                    THEN last_pvl.datetimeprel
            END
        ) AS dernier_prel_datetime_n02,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO3_NO2'
                    THEN last_pvl.datetimeprel
            END
        ) AS dernier_prel_datetime_no3_no2,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO3'
                    THEN last_pvl.valtraduite
            END
        ) AS valtraduite_no3,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO2'
                    THEN last_pvl.valtraduite
            END
        ) AS valtraduite_no2,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO3_NO2'
                    THEN last_pvl.valtraduite
            END
        ) AS valtraduite_no3_no2
    FROM
        last_pvl
    WHERE
        last_pvl.row_number = 1
    GROUP BY
        last_pvl.cdreseau,
        last_pvl.categorie
),

split_nitrites_with_ref AS (
    SELECT 
        split_nitrites.*,
        valeur_ref.limite_qualite_no3,
        valeur_ref.limite_qualite_no3_no2,
        valeur_ref.limite_qualite_no2,
        split_nitrites.valtraduite_no3 / 50
        + split_nitrites.valtraduite_no2 / 3 AS valtraduite_no3_no2_calc,
        DATE_DIFF(
                 'day',
                 LEAST(
                     split_nitrites.dernier_prel_datetime_n03,
                     split_nitrites.dernier_prel_datetime_n02,
                     split_nitrites.dernier_prel_datetime_no3_no2
                 ),
                 GREATEST(
                     split_nitrites.dernier_prel_datetime_n03,
                     split_nitrites.dernier_prel_datetime_n02,
                     split_nitrites.dernier_prel_datetime_no3_no2
                 )
             ) AS date_diff_count
    FROM split_nitrites
    CROSS JOIN valeur_ref

)

SELECT
    split_nitrites_with_ref.cdreseau,
    split_nitrites_with_ref.categorie,
    'dernier_prel' AS periode,
    split_nitrites_with_ref.dernier_prel_datetime,
    split_nitrites_with_ref.nb_parametres,
    split_nitrites_with_ref.valtraduite_no2,
    split_nitrites_with_ref.valtraduite_no3,
    split_nitrites_with_ref.valtraduite_no3_no2,
    split_nitrites_with_ref.valtraduite_no3_no2_calc,
    split_nitrites_with_ref.date_diff_count
FROM
    split_nitrites_with_ref  
WHERE
    cdreseau IN( '002000060','095000346','001000003','976003554','088002296','073002059','973000028','001000260','001000268','001000293')
ORDER BY 
    cdreseau

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

cdreseau,categorie,periode,dernier_prel_datetime,nb_parametres,valtraduite_no2,valtraduite_no3,valtraduite_no3_no2,valtraduite_no3_no2_calc,date_diff_count
001000003,nitrate,dernier_prel,2025-01-21 12:35:00,3,0.000,14.000,0.280,0.28,0
001000260,nitrate,dernier_prel,2024-12-06 12:02:00,3,0.000,2.400,0.050,0.048,0
001000268,nitrate,dernier_prel,2025-01-13 09:57:00,3,0.000,6.000,0.120,0.12,0
001000293,nitrate,dernier_prel,2025-01-28 11:29:00,3,0.000,3.700,0.070,0.07400000000000001,0
002000060,nitrate,dernier_prel,2024-10-18 08:50:00,3,0.000,59.300,0.000,1.186,0
073002059,nitrate,dernier_prel,2024-10-23 09:28:00,1,0.000,None,None,None,0
088002296,nitrate,dernier_prel,2024-09-27 12:00:00,1,0.000,None,None,None,0
095000346,nitrate,dernier_prel,2025-02-27 08:35:00,3,0.000,47.000,1.000,0.94,241
973000028,nitrate,dernier_prel,2024-12-02 11:30:00,2,0.000,0.000,None,0.0,0


In [9]:
%%sql --with int__resultats_nitrites_udi_dernier 
 SELECT 
   *
 FROM
   int__resultats_nitrites_udi_dernier
WHERE
    cdreseau IN( '002000060','095000346','001000003','976003554','088002296','073002059','973000028','001000260','001000268','001000293',)
ORDER BY 
    cdreseau

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

cdreseau,categorie,periode,dernier_prel_datetime,nb_parametres,resultat
001000003,nitrate,dernier_prel,2025-01-21 12:35:00,3,conforme
001000260,nitrate,dernier_prel,2024-12-06 12:02:00,3,conforme
001000268,nitrate,dernier_prel,2025-01-13 09:57:00,3,conforme
001000293,nitrate,dernier_prel,2025-01-28 11:29:00,3,conforme
002000060,nitrate,dernier_prel,2024-10-18 08:50:00,3,non_conforme
073002059,nitrate,dernier_prel,2024-10-23 09:28:00,1,donnee_manquante
088002296,nitrate,dernier_prel,2024-09-27 12:00:00,1,donnee_manquante
095000346,nitrate,dernier_prel,2025-02-27 08:35:00,3,non_conforme
973000028,nitrate,dernier_prel,2024-12-02 11:30:00,2,conforme


# Ajout d'une condition sur les dates de prélèvement/mesure

**Les règles choisies**
    
_Si on a les 3 paramètres disponibles **avec moins de 30 jours de différences**_
- Si Nitrates < 50 mg/L et nitrites < 0,5 mg/L et nitrate/50 + nitrites/3 < 1 mg/L  --> CONFORME
- Si Nitrates >= 50 mg/L et/ou nitrites >= 0,5 mg/L et/ou nitrate/50 + nitrites/3 >= 1 mg/L --> NON CONFORME


_Si on n'a pas  nitrate/50 + nitrites/3 mais qu'on a nitrate et  nitrites_<br>
==> on le calcule et on applique comme ci-dessus

- Si on n'a pas nitrate (no3) -->  donnee_manquante
- Si on n'a pas nitrites (no2) et que  nitrate >= 50 mg/L --> NON CONFORME
- Si on n'a pas nitrites (no2) et que  nitrate < 50 mg/L --> donnee_manquant 


In [10]:
%%sql
WITH
last_pvl AS (
    SELECT
        cdreseau,
        categorie,
        cdparametresiseeaux,
        limite_qualite,
        datetimeprel,
        valtraduite,
        ROW_NUMBER()
            OVER (
                PARTITION BY cdreseau, cdparametresiseeaux
                ORDER BY datetimeprel DESC
            )
            AS row_number
    FROM
       int__resultats_udi_communes
    WHERE
        categorie = 'nitrate'
        AND
        -- On garde les prélèvements de moins d'un an
        CURRENT_DATE - datetimeprel < INTERVAL 1 YEAR
),

valeur_ref AS (
SELECT 
    MAX(
            CASE
                WHEN
                    cdparametresiseeaux = 'NO3'
                    THEN limite_qualite
            END
        ) AS limite_qualite_no3,
        MAX(
            CASE
                WHEN
                    cdparametresiseeaux = 'NO3_NO2'
                    THEN limite_qualite
            END
        ) AS limite_qualite_no3_no2,
        MAX(
            CASE
                WHEN
                    cdparametresiseeaux = 'NO2'
                    THEN limite_qualite
            END
        ) AS limite_qualite_no2
FROM int__valeurs_de_reference
WHERE categorie_1='nitrate'    
),

split_nitrites AS (
    SELECT
        last_pvl.cdreseau,
        last_pvl.categorie,
        COUNT(DISTINCT last_pvl.cdparametresiseeaux) AS nb_parametres,
        MAX(last_pvl.datetimeprel) AS dernier_prel_datetime,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO3'
                    THEN last_pvl.datetimeprel
            END
        ) AS dernier_prel_datetime_n03,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO2'
                    THEN last_pvl.datetimeprel
            END
        ) AS dernier_prel_datetime_n02,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO3'
                    THEN last_pvl.valtraduite
            END
        ) AS valtraduite_no3,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO2'
                    THEN last_pvl.valtraduite
            END
        ) AS valtraduite_no2,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO3_NO2'
                    THEN last_pvl.valtraduite
            END
        ) AS valtraduite_no3_no2
    FROM
        last_pvl
    WHERE
        last_pvl.row_number = 1
    GROUP BY
        last_pvl.cdreseau,
        last_pvl.categorie
),

split_nitrites_with_ref AS (
    SELECT 
        DATE_DIFF( 'day',split_nitrites.dernier_prel_datetime_n03,split_nitrites.dernier_prel_datetime_n02) AS  date_diff,
        split_nitrites.dernier_prel_datetime_n03,
        split_nitrites.dernier_prel_datetime_n02,
        split_nitrites.cdreseau,
        split_nitrites.categorie,
        split_nitrites.nb_parametres,
        split_nitrites.dernier_prel_datetime,
        IF (  DATE_DIFF( 'day',split_nitrites.dernier_prel_datetime_n03,split_nitrites.dernier_prel_datetime_n02) >30,
              NULL, valtraduite_no3) AS valtraduite_no3,
        IF (  DATE_DIFF( 'day',split_nitrites.dernier_prel_datetime_n02,split_nitrites.dernier_prel_datetime_n03) >30,
              NULL, valtraduite_no2) AS valtraduite_no2,
        COALESCE(
            split_nitrites.valtraduite_no3_no2,
            (
                split_nitrites.valtraduite_no3 / 50
                + split_nitrites.valtraduite_no2 / 3
            )
        ) AS valtraduite_no3_no2,
    
        DATE_DIFF(
                 'day',
                 LEAST(
                     split_nitrites.dernier_prel_datetime_n03,
                     split_nitrites.dernier_prel_datetime_n02
                 ),
                 GREATEST(
                     split_nitrites.dernier_prel_datetime_n03,
                     split_nitrites.dernier_prel_datetime_n02
                 )
             ) AS date_diff_count
    FROM split_nitrites
    CROSS JOIN valeur_ref

)

SELECT
*
FROM
    split_nitrites_with_ref  
WHERE
     split_nitrites_with_ref.date_diff_count > 30
    AND valtraduite_no3 > 50
ORDER BY 
    cdreseau

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

date_diff,dernier_prel_datetime_n03,dernier_prel_datetime_n02,cdreseau,categorie,nb_parametres,dernier_prel_datetime,valtraduite_no3,valtraduite_no2,valtraduite_no3_no2,date_diff_count
-45,2024-11-25 09:29:00,2024-10-11 12:55:00,002000074,nitrate,3,2024-11-25 09:29:00,56.900,None,0.0,45
-61,2024-12-23 13:09:00,2024-10-23 08:29:00,002001241,nitrate,3,2024-12-23 13:09:00,52.500,None,0.0,61
-141,2025-01-28 09:24:00,2024-09-09 09:09:00,008000002,nitrate,2,2025-01-28 09:24:00,52.000,None,1.04,141
-161,2025-02-07 10:38:00,2024-08-30 13:21:00,008000546,nitrate,2,2025-02-07 10:38:00,76.300,None,1.526,161
-37,2025-02-26 11:59:00,2025-01-20 13:30:00,008000679,nitrate,3,2025-02-26 11:59:00,53.700,None,0.0,37
-100,2025-02-12 11:38:00,2024-11-04 11:23:00,008000681,nitrate,2,2025-02-12 11:38:00,54.900,None,1.0979999999999999,100
-142,2025-02-19 12:10:00,2024-09-30 08:32:00,008001171,nitrate,2,2025-02-19 12:10:00,50.500,None,1.01,142
-74,2025-02-18 10:16:00,2024-12-06 08:10:00,010000072,nitrate,3,2025-02-18 10:16:00,56.000,None,1.1,74
-175,2025-02-18 12:24:00,2024-08-27 10:03:00,010000122,nitrate,3,2025-02-18 12:24:00,64.000,None,1.1,175
-98,2025-02-24 11:25:00,2024-11-18 11:22:00,010000125,nitrate,3,2025-02-24 11:25:00,72.000,None,1.4,98


In [11]:
%%sql --save int__resultats_nitrites_udi_dernier_dateCheck
WITH
last_pvl AS (
    SELECT
        cdreseau,
        categorie,
        cdparametresiseeaux,
        limite_qualite,
        datetimeprel,
        valtraduite,
        ROW_NUMBER()
            OVER (
                PARTITION BY cdreseau, cdparametresiseeaux
                ORDER BY datetimeprel DESC
            )
            AS row_number
    FROM
        int__resultats_udi_communes
    WHERE
        categorie = 'nitrate'
        AND
        -- On garde les prélèvements de moins d'un an
        CURRENT_DATE - datetimeprel < INTERVAL 1 YEAR
),

valeur_ref AS (
    SELECT
        MAX(
            CASE
                WHEN
                    cdparametresiseeaux = 'NO3'
                    THEN limite_qualite
            END
        ) AS limite_qualite_no3,
        MAX(
            CASE
                WHEN
                    cdparametresiseeaux = 'NO3_NO2'
                    THEN limite_qualite
            END
        ) AS limite_qualite_no3_no2,
        MAX(
            CASE
                WHEN
                    cdparametresiseeaux = 'NO2'
                    THEN limite_qualite
            END
        ) AS limite_qualite_no2
    FROM
        int__valeurs_de_reference
    WHERE
        categorie_1 = 'nitrate'
),

split_nitrites AS (
    SELECT
        last_pvl.cdreseau,
        last_pvl.categorie,
        COUNT(DISTINCT last_pvl.cdparametresiseeaux) AS nb_parametres,
        MAX(last_pvl.datetimeprel) AS dernier_prel_datetime,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO3'
                    THEN last_pvl.valtraduite
            END
        ) AS valtraduite_no3,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO2'
                    THEN last_pvl.valtraduite
            END
        ) AS valtraduite_no2,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO3_NO2'
                    THEN last_pvl.valtraduite
            END
        ) AS valtraduite_no3_no2,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO3'
                    THEN last_pvl.datetimeprel
            END
        ) AS dernier_prel_datetime_n03,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO2'
                    THEN last_pvl.datetimeprel
            END
        ) AS dernier_prel_datetime_n02
    FROM
        last_pvl
    WHERE
        last_pvl.row_number = 1
    GROUP BY
        last_pvl.cdreseau,
        last_pvl.categorie
),

split_nitrites_with_ref AS (
    SELECT
        split_nitrites.cdreseau,
        split_nitrites.categorie,
        split_nitrites.nb_parametres,
        split_nitrites.dernier_prel_datetime,
        valeur_ref.limite_qualite_no3,
        valeur_ref.limite_qualite_no2,
        valeur_ref.limite_qualite_no3_no2,
        split_nitrites.valtraduite_no3_no2,
        -- Si les mesures sont espacées de plus de 30 jours 
        -- on garde que la plus récente uniquement 
        IF(
            DATE_DIFF(
                'day',
                split_nitrites.dernier_prel_datetime_n03,
                split_nitrites.dernier_prel_datetime_n02
            ) > 30,
            NULL,
            split_nitrites.valtraduite_no3
        ) AS valtraduite_no3,
        IF(
            DATE_DIFF(
                'day',
                split_nitrites.dernier_prel_datetime_n02,
                split_nitrites.dernier_prel_datetime_n03
            ) > 30,
            NULL,
            split_nitrites.valtraduite_no2
        ) AS valtraduite_no2
    FROM
        split_nitrites
    CROSS JOIN
        valeur_ref
)

SELECT
    split_nitrites_with_ref.cdreseau,
    split_nitrites_with_ref.categorie,
    'dernier_prel' AS periode,
    split_nitrites_with_ref.dernier_prel_datetime,
    split_nitrites_with_ref.nb_parametres,
    CASE
        WHEN
            split_nitrites_with_ref.valtraduite_no3
            < split_nitrites_with_ref.limite_qualite_no3
            AND split_nitrites_with_ref.valtraduite_no2
            < split_nitrites_with_ref.limite_qualite_no2
            -- Si on n'a pas valtraduite_no3_no2, on le calcule
            -- si l'une des valeurs de la somme est NULL le total sera NULL 
            AND COALESCE(
                split_nitrites_with_ref.valtraduite_no3_no2,
                (
                    split_nitrites_with_ref.valtraduite_no3 / 50
                    + split_nitrites_with_ref.valtraduite_no2 / 3
                )
            )
            < split_nitrites_with_ref.limite_qualite_no3_no2
            THEN 'conforme'
        WHEN
            split_nitrites_with_ref.valtraduite_no3
            >= split_nitrites_with_ref.limite_qualite_no3
            OR split_nitrites_with_ref.valtraduite_no2
            >= split_nitrites_with_ref.limite_qualite_no2
            -- Si on n'a pas valtraduite_no3_no2, on le calcule
            -- si l'une des valeurs de la somme est NULL le total sera NULL 
            OR COALESCE(
                split_nitrites_with_ref.valtraduite_no3_no2,
                (
                    split_nitrites_with_ref.valtraduite_no3 / 50
                    + split_nitrites_with_ref.valtraduite_no2 / 3
                )
            )
            >= split_nitrites_with_ref.limite_qualite_no3_no2
            THEN 'non_conforme'
        WHEN
            split_nitrites_with_ref.valtraduite_no3 IS NULL
            OR (
                split_nitrites_with_ref.valtraduite_no2 IS NULL
                AND split_nitrites_with_ref.valtraduite_no3
                < split_nitrites_with_ref.limite_qualite_no3
            )
            THEN 'donnee_manquante'
        ELSE 'error'
    END AS resultat
FROM
    split_nitrites_with_ref


Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

cdreseau,categorie,periode,dernier_prel_datetime,nb_parametres,resultat
001000241,nitrate,dernier_prel,2025-02-10 11:04:00,3,conforme
001000304,nitrate,dernier_prel,2025-02-11 10:10:00,3,conforme
001000313,nitrate,dernier_prel,2024-11-27 11:00:00,3,conforme
001000387,nitrate,dernier_prel,2025-02-13 10:48:00,3,donnee_manquante
001000427,nitrate,dernier_prel,2025-01-24 11:15:00,3,conforme
001000464,nitrate,dernier_prel,2025-01-16 09:17:00,3,conforme
001000490,nitrate,dernier_prel,2024-09-27 11:53:00,3,conforme
001000585,nitrate,dernier_prel,2025-02-21 09:15:00,3,conforme
001000824,nitrate,dernier_prel,2024-10-18 09:54:00,3,conforme
001000865,nitrate,dernier_prel,2025-02-11 09:27:00,3,conforme


In [12]:
%%sql --with int__resultats_nitrites_udi_dernier_dateCheck
SELECT 
    resultat, COUNT(*)
FROM 
    int__resultats_nitrites_udi_dernier_dateCheck
GROUP BY 1

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

resultat,count_star()
non_conforme,269
conforme,17338
donnee_manquante,4528


In [13]:
%%sql --with int__resultats_nitrites_udi_dernier_dateCheck
SELECT 
    * 
FROM 
    int__resultats_nitrites_udi_dernier_dateCheck
WHERE 
    resultat='error'

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

cdreseau,categorie,periode,dernier_prel_datetime,nb_parametres,resultat


In [14]:
%%sql --with int__resultats_nitrites_udi_dernier_dateCheck 
 SELECT 
   *
 FROM
   int__resultats_nitrites_udi_dernier_dateCheck
WHERE
    cdreseau IN( '002000060','095000346','001000003','976003554','088002296','073002059','973000028','001000260','001000268','001000293',
    '001000258','001000387','008000681','008000546')
ORDER BY 
    cdreseau

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

cdreseau,categorie,periode,dernier_prel_datetime,nb_parametres,resultat
001000003,nitrate,dernier_prel,2025-01-21 12:35:00,3,conforme
001000258,nitrate,dernier_prel,2025-01-21 09:45:00,3,donnee_manquante
001000260,nitrate,dernier_prel,2024-12-06 12:02:00,3,conforme
001000268,nitrate,dernier_prel,2025-01-13 09:57:00,3,conforme
001000293,nitrate,dernier_prel,2025-01-28 11:29:00,3,conforme
001000387,nitrate,dernier_prel,2025-02-13 10:48:00,3,donnee_manquante
002000060,nitrate,dernier_prel,2024-10-18 08:50:00,3,non_conforme
008000546,nitrate,dernier_prel,2025-02-07 10:38:00,2,non_conforme
008000681,nitrate,dernier_prel,2025-02-12 11:38:00,2,non_conforme
073002059,nitrate,dernier_prel,2024-10-23 09:28:00,1,donnee_manquante


In [15]:
%%sql --with int__resultats_nitrites_udi_dernier_dateCheck 
SELECT
    'dernier relevé' AS periode,
    cdreseau,
    categorie,
    resultat,
FROM
    int__resultats_nitrites_udi_dernier_dateCheck
WHERE
    (
        cdreseau = '001000003'
        AND categorie = 'nitrate'
        AND dernier_prel_datetime = '2024-12-26 09:08:00'
        AND resultat != 'conforme'
    )
    OR
    (
        cdreseau = '001000260'
        AND categorie = 'nitrate'
        AND dernier_prel_datetime = '2024-12-06 12:02:00'
        AND resultat != 'conforme'
    )
    OR
    (
        cdreseau = '001000268'
        AND categorie = 'nitrate'
        AND dernier_prel_datetime = '2024-11-28 08:25:00'
        AND resultat != 'conforme'
    )
    OR
    (
        cdreseau = '002000060'
        AND categorie = 'nitrate'
        AND dernier_prel_datetime = '2024-10-18 08:50:00'
        AND resultat != 'non_conforme'
    )
    OR
    (
        cdreseau = '073002059'
        AND categorie = 'nitrate'
        AND dernier_prel_datetime = '2024-10-23 09:28:00'
        AND resultat != 'donnee_manquante'
        AND nb_parametres = 1
    )
    OR
    (
        cdreseau = '088002296'
        AND categorie = 'nitrate'
        AND dernier_prel_datetime = '2024-09-27 12:00:00'
        AND resultat != 'donnee_manquante'
        AND nb_parametres = 1
    )
    OR
    (
        cdreseau = '095000346'
        AND categorie = 'nitrate'
        AND dernier_prel_datetime = '2024-12-17 08:55:00'
        AND resultat != 'non_conforme'
    )
    OR
    (
        cdreseau = '973000028'
        AND categorie = 'nitrate'
        AND dernier_prel_datetime = '2024-12-17 08:55:00'
        AND resultat != 'conforme'
        AND nb_parametres = 2
    )
    OR
    (
        cdreseau = '001000258'
        AND categorie = 'nitrate'
        AND dernier_prel_datetime = '2025-01-21 09:45:00'
        AND resultat != 'donnee_manquante'
        AND nb_parametres = 2
    )
    OR
    (
        cdreseau = '001000387'
        AND categorie = 'nitrate'
        AND dernier_prel_datetime = '2025-02-13 10:48:00'
        AND resultat != 'conforme'
        AND nb_parametres = 2
    )
    OR
    (
        cdreseau = '008000681'
        AND categorie = 'nitrate'
        AND dernier_prel_datetime = '2025-02-12 11:38:00'
        AND resultat != 'non_conforme'
        AND nb_parametres = 2
    )
    OR
    (
        cdreseau = '008000546'
        AND categorie = 'nitrate'
        AND dernier_prel_datetime = '2025-02-07 10:38:00'
        AND resultat != 'non_conforme'
        AND nb_parametres = 2
    )

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

periode,cdreseau,categorie,resultat
